In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from databroker import Broker
import os

import pandas as pd

### Load databroker database

In [ ]:
path = '/opt/papermill/results'
uid = 'b88a8255-82a6-4ddc-8565-2a5c02551999'
x_roi = 262
y_roi1 = 55
y_roi2 = 71
y_roi3 = 88
wdt = 13
hgt = 10

In [ ]:
db = Broker.named('jpls')
h = db[uid]
print(h)

images = np.asarray(list(h.data('pilatus100k_image')))
attenuation = list(h.data('attenuation'))

print(h.start['uid'])
print(h.stream_names)
print(images.shape)
print(attenuation)
h.table()

In [ ]:
exp_time = h.start['exposure_time']
ener = h.start['energy']
sam = h.start['sample_name']

attenu = list(h.data('attenuation'))
i0 = list(h.data('quadem_current3_mean_value'))
alpha_i = list(h.data('geo_alpha'))

images = np.asarray(list(h.data('pilatus100k_image')))

x_roi = 262
y_roi1 = 55
y_roi2 = 71
y_roi3 = 88
wdt = 13
hgt = 10

int_roi1 = np.zeros(np.shape(alpha_i))
int_roi2 = np.zeros(np.shape(alpha_i))
int_roi3 = np.zeros(np.shape(alpha_i))
qz = np.zeros(np.shape(alpha_i))
ref_sub = np.zeros(np.shape(alpha_i))

lbda = 12.39842/(ener/1000)


for i, (att, izer, alph_i, img) in enumerate(zip(attenu, i0, alpha_i, images)):
    print(np.shape(img))
    im = img/(att*izer)
    int_roi1[i] = im[y_roi1:y_roi1+hgt, x_roi:x_roi+wdt].sum()
    int_roi2[i] = im[y_roi2:y_roi2+hgt, x_roi:x_roi+wdt].sum()
    int_roi3[i] = im[y_roi3:y_roi3+hgt, x_roi:x_roi+wdt].sum()
    
    qz[i] = 4*np.pi*np.sin(alph_i/180*np.pi)/lbda
    
ref_sub = int_roi2 - (int_roi1 + int_roi3)*0.5
df = pd.DataFrame({'qz': qz, 'ref_sub': ref_sub})

plt.figure()
plt.plot(qz, ref_sub)
plt.show()

uid_0 = uid.split('-')[0]
df.to_csv(os.path.join(path, f'test_reflection-{sam}-{uid_0}.txt'), sep='\t')
#np.savetxt(os.path.join(path, 'test_reflection.txt'), np.vstack((qz, ref_sub)).T)


In [ ]:
# save a file


In [ ]:
qz
lbda
ener